In [1]:
import tensorflow as tf
import numpy as np
from custom_layers import *
from worker import Worker

In [2]:
coordinator = tf.train.Coordinator()

hidden_layer_size = 32
output_size = 10
global_model = FullyConnectedModel(
                output_size=output_size,
                hidden_layer_size=hidden_layer_size,
            )
#_ = global_model([global_model.inputs['img'], global_model.inputs['mask']])
#print(global_model.accumulators)
fake_img = np.zeros((1, 28, 28))
fake_mask = np.zeros((1, hidden_layer_size))
print("Init call: ", global_model([fake_img, fake_mask]))
'''
for layer in global_model.layers:
    layer.trainable = False
'''
global_model.summary()
print("weights: ", global_model.get_weights())
print("accumulators: ", global_model.accumulators)
global_model.init_accumulators()
print("accumulators: ", global_model.accumulators)
print("weights: ", global_model.get_weights())
print("trainable: ", global_model.non_trainable_weights)
#global_model.copy_weights(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Init call:  tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)
Model: "fully_connected_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_flatte multiple                  0         
_________________________________________________________________
fully_connected_model_encode multiple                  25120     
____________________________________________________________

In [3]:
global_model.trainable_weights

[<tf.Variable 'fully_connected_model/fully_connected_model_encoder/kernel:0' shape=(784, 32) dtype=float32, numpy=
 array([[-0.02102112,  0.04726418,  0.06960919, ...,  0.07084014,
         -0.02675245, -0.08079204],
        [ 0.02426789, -0.0247478 , -0.05079987, ...,  0.04477855,
          0.02703669,  0.0258103 ],
        [ 0.07048052, -0.00606384,  0.00505009, ...,  0.00376936,
          0.02053283,  0.06786665],
        ...,
        [-0.07612689, -0.07649571, -0.00937118, ...,  0.04724241,
         -0.079721  , -0.07272233],
        [-0.08546587, -0.03546625,  0.03747183, ..., -0.04969054,
          0.04259578,  0.00474735],
        [-0.0701976 , -0.00927283, -0.07002833, ..., -0.00193752,
          0.03743515,  0.06560641]], dtype=float32)>,
 <tf.Variable 'fully_connected_model/fully_connected_model_encoder/bias:0' shape=(32,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [4]:
num_models = 3
models = {}
for idx in range(num_models):
    model = FullyConnectedModel(
            output_size=output_size,
            hidden_layer_size=hidden_layer_size,
            #parent_model=global_model,
        )
    #_ = model([model.inputs['img'], model.inputs['mask']])
    print(model([fake_img, fake_mask]))
    models[idx] = model
    #model.set_parent_model(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting

In [5]:
for idx in range(num_models):
    t = Worker(args=idx, kwargs=
                {
                    'coordinator': coordinator,
                    'global_model': global_model,
                    'model': models[idx],
                    'hidden_layer_size': hidden_layer_size,
                    'output_size': output_size,
                    'learning_rate': 1e-4,
                    'dropout_rate': 1-0.5,
                    'batch_size': 196,
                    'epochs': 10,
                    'update_batches': 100
                }
    )
    t.start()


Model: "fully_connected_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_1_flat multiple                  0         
_________________________________________________________________
fully_connected_model_1_enco multiple                  25120     
_________________________________________________________________
fully_connected_model_1_Cust multiple                  0         
_________________________________________________________________
fully_connected_model_1_outp multiple                  330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________
None
Model: "fully_connected_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_2_flat multiple                  0         
_